In [8]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [9]:
import pandas as pd
df_annotated = pd.read_excel('/content/drive/My Drive/Kuliah MTI/Anmedsos/labeling_dataset.xlsx', sheet_name='Sheet1')

In [20]:
df_unannotated = pd.read_excel('/content/drive/My Drive/Kuliah MTI/Anmedsos/unlabeled_dataset.xlsx', sheet_name='Sheet1')

In [21]:
df_annotated.head()

,text,created_at,event_sentiment,circuit_sentiment
0,"""Five years ago, I had my most serious acciden...",2025-07-20 14.40.42,neutral,neutral
1,Legenda balap motor Valentino Rossi tiba di Ja...,2025-09-30 20.54.12,positive,neutral
2,Menyusuri Sejarah Motorsport Indonesia di Muse...,2025-10-06 19.08.19,neutral,neutral
3,Menteri Pemuda dan Olahraga Erick Thohir menye...,2025-10-05 12.39.53,positive,positive
4,Momen Pacu Jalur Rayyan Arkan Dikha Guncang Ma...,2025-10-04 20.21.14,positive,neutral


In [22]:
df_unannotated.head()

,text,created_at
0,Menggenang marco simoncelli:\nPrompt :👇☠️\nBua...,2025-10-03 22.34.55
1,"Kemarin liat berita Mandalika GP, tiketnya gak...",2025-10-03 15.36.02
2,Ternyata MotoGP Mandalika barengan sama F1 Sin...,2025-10-04 17.11.57
3,Dua tahun berturut-turut nonton MotoGP Mandali...,2025-10-07 18.15.24
4,"Welcome back to Paradise!\nMotoGP Mandalika, p...",2024-09-28 07.02.23


In [7]:
pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 86.6 MB/s eta 0:00:00


In [8]:
pip install tensorflow

In [8]:
pip install Sastrawi

In [18]:
pip install pandarallel

  Preparing metadata (setup.py) ... done
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16674 sha256=7cc267dab983647621fae8c74b2ce798f34b1426a3f5965d6c9917c6b79d3cff
  Stored in directory: /root/.cache/pip/wheels/46/f9/0d/40c9cd74a7cb8dc8fe57e8d6c3c19e2c730449c0d3f2bf66b5
Successfully built pandarallel


In [23]:
# -*- coding: utf-8 -*-
"""
Skrip Python KomprehensIF untuk Analisis Sentimen Berbasis Aspek (Versi Diperluas)
Tujuan: Menganalisis sentimen wisatawan terhadap event MotoGP di Sirkuit Mandalika.
Metodologi:
1.  Prapemrosesan Data: Tokenisasi, Stemming (Sastrawi), POS Tagging (NLTK-EN),
    Filter Kata Benda (Nouns), Stopword Removal (Sastrawi-ID + NLTK-EN).
2.  Ekstraksi Fitur DL: Word2Vec (dilatih pada korpus).
3.  Ekstraksi Fitur ML: Word2Vec (Rata-rata), Bag of Words (BoW), dan TF-IDF.
4.  Pelatihan Model Deep Learning: CNN, LSTM, RNN (menggunakan Word2Vec).
5.  Pelatihan Model Machine Learning: Logistic Regression, Decision Tree, Random Forest, XGBoost
    (menggunakan W2V-Avg, BoW, dan TF-IDF).
6.  Validasi Silang (3-Fold Cross-Validation) untuk semua model.
7.  Implementasi model terbaik untuk memprediksi sentimen pada data sisa (unlabeled).

PENTING: Skrip ini berasumsi dua pandas DataFrames bernama 'df_annotated'
dan 'df_unannotated' sudah ada di environment Anda.
"""

# Import library yang dibutuhkan
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover import StopWordRemoverFactory
from pandarallel import pandarallel
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, LSTM, SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import warnings

# Mute warnings dari sklearn dan xgboost
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# --- 1. PERSIAPAN AWAL & DOWNLOAD DEPENDENSI NLTK ---
try:
    stopwords.words('english') # Hanya NLTK English stopwords yang kita perlukan
    print("NLTK stopwords (English) sudah ada.")
except LookupError:
    print("Mengunduh NLTK stopwords (English)...")
    nltk.download('stopwords')

try:
    nltk.data.find('tokenizers/punkt_tab')
    print("NLTK punkt_tab sudah ada.")
except LookupError:
    print("Mengunduh 'punkt_tab' yang hilang...")
    nltk.download('punkt_tab')

try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
    print("NLTK POS Tagger (averaged_perceptron_tagger) sudah ada.")
except LookupError:
    print("Mengunduh NLTK POS Tagger...")
    nltk.download('averaged_perceptron_tagger')

try:
    nltk.data.find('taggers/averaged_perceptron_tagger_eng')
    print("NLTK POS Tagger (averaged_perceptron_tagger_eng) sudah ada.")
except LookupError:
    print("Mengunduh NLTK POS Tagger (sub-modul 'eng')...")
    nltk.download('averaged_perceptron_tagger_eng')

print("Persiapan NLTK selesai.")

# --- 2. FUNGSI PRAPEMROSESAN DATA (DENGAN STEMMING & POS TAGGING) ---
def preprocess_text_with_pos(text, stemmer, combined_stopwords):
    """
    Fungsi untuk membersihkan teks, melakukan stemming (Sastrawi),
    POS tagging (NLTK-EN), memfilter kata benda (nouns),
    dan menghapus stopwords (Sastrawi-ID + NLTK-EN).

    Mengembalikan:
    1. List token kata benda yang sudah bersih.
    2. String gabungan dari token kata benda yang sudah bersih.
    """
    if not isinstance(text, str):
        return [], ""

    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)

    # 1. Tokenisasi
    tokens = word_tokenize(text)

    # 2. Stemming (Sastrawi)
    # Diterapkan sebelum POS Tagging untuk menormalkan kata-kata Bahasa Indonesia.
    # Stemmer Sastrawi cukup pintar untuk mengabaikan kata-kata Bahasa Inggris.
    stemmed_tokens = [stemmer.stem(word) for word in tokens]

    # 3. POS Tagging (NLTK - English)
    # Tagger ini akan bekerja pada token yang sudah di-stem.
    pos_tags = nltk.pos_tag(stemmed_tokens)

    # 4. Filter Kata Benda (Nouns)
    # NN (noun, singular), NNS (noun, plural), NNP (proper noun, singular), NNPS (proper noun, plural)
    noun_tokens = [word for word, tag in pos_tags if tag in ('NN', 'NNS', 'NNP', 'NNPS')]

    # 5. Hapus Stopwords
    # Menggunakan daftar gabungan yang sudah disiapkan sebelumnya
    filtered_nouns = [word for word in noun_tokens if word not in combined_stopwords and len(word) > 2]

    return filtered_nouns, " ".join(filtered_nouns)


# --- 3. EKSTRAKSI FITUR (WORD EMBEDDING) ---
def train_word2vec(corpus_tokens, vector_size=100, window=5, min_count=2):
    """Melatih model Word2Vec dari korpus teks multibahasa (list of lists)."""
    model = Word2Vec(corpus_tokens, vector_size=vector_size, window=window, min_count=min_count, workers=4)
    return model

def create_embedding_matrix_from_word2vec(word2vec_model, word_index):
    """Membuat embedding matrix dari model Word2Vec yang sudah dilatih (untuk Keras)."""
    embedding_dim = word2vec_model.vector_size
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in word2vec_model.wv:
            embedding_matrix[i] = word2vec_model.wv[word]
    return embedding_matrix

def create_document_vectors(corpus_tokenized, model, embedding_dim):
    """
    Membuat vektor dokumen rata-rata untuk model ML klasik.
    """
    vectors = []
    for doc_tokens in corpus_tokenized:
        doc_vec = np.zeros(embedding_dim)
        count = 0
        for word in doc_tokens:
            if word in model.wv:
                doc_vec += model.wv[word]
                count += 1
        if count > 0:
            doc_vec /= count  # Rata-rata vektor
        vectors.append(doc_vec)
    return np.array(vectors)

# --- 4. PEMBUATAN MODEL ---
# (Fungsi create_cnn_model, create_lstm_model, create_rnn_model tetap sama)
def create_cnn_model(embedding_matrix, max_len):
    model = Sequential([
        Embedding(input_dim=embedding_matrix.shape[0],
                  output_dim=embedding_matrix.shape[1],
                  weights=[embedding_matrix],
                  input_length=max_len,
                  trainable=False),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_lstm_model(embedding_matrix, max_len):
    model = Sequential([
        Embedding(input_dim=embedding_matrix.shape[0],
                  output_dim=embedding_matrix.shape[1],
                  weights=[embedding_matrix],
                  input_length=max_len,
                  trainable=False),
        LSTM(64, dropout=0.2, recurrent_dropout=0.2),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_rnn_model(embedding_matrix, max_len):
    model = Sequential([
        Embedding(input_dim=embedding_matrix.shape[0],
                  output_dim=embedding_matrix.shape[1],
                  weights=[embedding_matrix],
                  input_length=max_len,
                  trainable=False),
        SimpleRNN(64),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# --- 5. PROSES UTAMA (MAIN WORKFLOW) ---
if __name__ == '__main__':

    # --- TAMBAHAN BARU: Inisialisasi Sastrawi & Stopwords Gabungan ---
    print("\n[INFO] Mempersiapkan Sastrawi Stemmer dan Stopword Remover...")
    # 1. Stemmer
    stemmer_factory = StemmerFactory()
    stemmer = stemmer_factory.create_stemmer()

    # 2. Stopwords
    stopword_factory = StopWordRemoverFactory.StopWordRemoverFactory()
    sastrawi_stopwords = set(stopword_factory.get_stop_words())
    nltk_english_stopwords = set(stopwords.words('english'))
    COMBINED_STOPWORDS = sastrawi_stopwords.union(nltk_english_stopwords)
    print("Sastrawi dan stopwords gabungan siap.")
    # --- AKHIR TAMBAHAN BARU ---

    # --- TAMBAHAN BARU: Inisialisasi Pandarallel ---
    # Menggunakan semua inti CPU yang tersedia
    pandarallel.initialize(progress_bar=True)

    # --- A. Muat dan Proses Data ---
    if 'df_annotated' not in locals() or 'df_unannotated' not in locals():
        print("="*50)
        print("ERROR: Pastikan 'df_annotated' dan 'df_unannotated' sudah dimuat.")
        print("Contoh: df_annotated = pd.read_csv('data_annotated_30persen.csv')")
        print("="*50)
        raise NameError("'df_annotated' or 'df_unannotated' not defined.")

    print(f"Jumlah data beranotasi: {len(df_annotated)}")
    print(f"Jumlah data belum beranotasi: {len(df_unannotated)}")


    # --- PERUBAHAN: Menerapkan prapemrosesan baru dengan Sastrawi ---
    print("[INFO] Menerapkan prapemrosesan (Stemming, POS Tag, Filter Nouns, Stopwords)...")
    print("Memproses df_annotated (mungkin butuh waktu)...")
    # Terapkan pada data beranotasi (MENGGUNAKAN parallel_apply)
    df_annotated[['cleaned_tokens_nouns', 'cleaned_text_nouns']] = \
        df_annotated['text'].parallel_apply(lambda x: pd.Series(preprocess_text_with_pos(x, stemmer, COMBINED_STOPWORDS)))

    print("Memproses df_unannotated (mungkin butuh waktu)...")
    # Terapkan pada data belum beranotasi (MENGGUNAKAN parallel_apply)
    df_unannotated[['cleaned_tokens_nouns', 'cleaned_text_nouns']] = \
        df_unannotated['text'].parallel_apply(lambda x: pd.Series(preprocess_text_with_pos(x, stemmer, COMBINED_STOPWORDS)))

    # Gabungkan semua teks (string) yang sudah bersih untuk vectorizer ML
    all_corpus_text_nouns = pd.concat([df_annotated['cleaned_text_nouns'], df_unannotated['cleaned_text_nouns']], ignore_index=True)
    # Gabungkan semua token (list) yang sudah bersih untuk Word2Vec
    all_corpus_tokens_nouns = pd.concat([df_annotated['cleaned_tokens_nouns'], df_unannotated['cleaned_tokens_nouns']], ignore_index=True).tolist()
    # --- AKHIR PERUBAHAN ---

    target_aspect = 'event_sentiment'

    sentiment_mapping = {'positive': 0, 'neutral': 1, 'negative': 2}
    df_annotated[target_aspect] = df_annotated[target_aspect].map(sentiment_mapping)
    df_annotated = df_annotated.dropna(subset=[target_aspect])

    # --- B. Persiapan Data (Label dan Fitur) ---

    # 1. Label
    y_categorical = tf.keras.utils.to_categorical(df_annotated[target_aspect].astype(int), num_classes=3)
    y_1d = df_annotated[target_aspect].astype(int) # Untuk model ML

    # 2. Fitur untuk Deep Learning (Tokenized & Padded Sequences)
    tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
    tokenizer.fit_on_texts(all_corpus_text_nouns)
    word_index = tokenizer.word_index
    sequences = tokenizer.texts_to_sequences(df_annotated['cleaned_text_nouns'])
    max_len = 100
    X_padded = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

    # Split data DL
    X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(
        X_padded, y_categorical, test_size=0.2, random_state=42, stratify=y_1d
    )

    # 3. Fitur untuk Machine Learning
    X_train_text_ml, X_test_text_ml, y_train_ml, y_test_ml = train_test_split(
        df_annotated, y_1d, test_size=0.2, random_state=42, stratify=y_1d
    )

    # --- C. Persiapan Model Embedding (Word2Vec) ---
    print("\n--- Menyiapkan Model Embedding (Word2Vec) ---")
    w2v_model = train_word2vec(all_corpus_tokens_nouns)
    print("Model Word2Vec selesai dilatih.")

    # --- D. Persiapan Fitur Set untuk ML ---
    print("\n--- Mempersiapkan Fitur Set Machine Learning ---")
    ml_feature_sets = {}

    # Fitur 1: Word2Vec (Rata-rata)
    embedding_dim = w2v_model.vector_size
    X_train_ml_w2v = create_document_vectors(X_train_text_ml['cleaned_tokens_nouns'], w2v_model, embedding_dim)
    X_test_ml_w2v = create_document_vectors(X_test_text_ml['cleaned_tokens_nouns'], w2v_model, embedding_dim)
    ml_feature_sets['Word2Vec_Avg'] = (X_train_ml_w2v, X_test_ml_w2v)
    print("Fitur Word2Vec (Avg) siap.")

    # Fitur 2: Bag of Words (BoW)
    bow_vectorizer = CountVectorizer(max_features=5000)
    X_train_ml_bow = bow_vectorizer.fit_transform(X_train_text_ml['cleaned_text_nouns'])
    X_test_ml_bow = bow_vectorizer.transform(X_test_text_ml['cleaned_text_nouns'])
    ml_feature_sets['BoW'] = (X_train_ml_bow, X_test_ml_bow)
    print("Fitur Bag of Words (BoW) siap.")

    # Fitur 3: TF-IDF
    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    X_train_ml_tfidf = tfidf_vectorizer.fit_transform(X_train_text_ml['cleaned_text_nouns'])
    X_test_ml_tfidf = tfidf_vectorizer.transform(X_test_text_ml['cleaned_text_nouns'])
    ml_feature_sets['TFIDF'] = (X_train_ml_tfidf, X_test_ml_tfidf)
    print("Fitur TF-IDF siap.")

    y_train_ml_np = y_train_ml.to_numpy()
    y_test_ml_np = y_test_ml.to_numpy()

    # --- E. Training & Evaluasi Model ---
    print("\n--- Memulai Pelatihan dan Evaluasi Model ---")

    models_to_evaluate_dl = {'CNN': create_cnn_model, 'LSTM': create_lstm_model, 'RNN': create_rnn_model}
    models_to_evaluate_ml = {
        'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
        'DecisionTree': DecisionTreeClassifier(random_state=42),
        'RandomForest': RandomForestClassifier(random_state=42),
        'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
    }

    results = {}

    # --- Loop Pelatihan Model Deep Learning (Hanya Word2Vec) ---
    embedding_matrix = create_embedding_matrix_from_word2vec(w2v_model, word_index)

    for model_name, model_creator in models_to_evaluate_dl.items():
        print(f"\nMelatih model {model_name} dengan embedding Word2Vev...")
        kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        fold_accuracies = []

        for train, val in kfold.split(X_train_dl, np.argmax(y_train_dl, axis=1)):
            model = model_creator(embedding_matrix, max_len)
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
            model.fit(X_train_dl[train], y_train_dl[train],
                      epochs=20, batch_size=32,
                      validation_data=(X_train_dl[val], y_train_dl[val]),
                      callbacks=[early_stopping], verbose=0)
            _, acc = model.evaluate(X_train_dl[val], y_train_dl[val], verbose=0)
            fold_accuracies.append(acc)

        avg_cv_accuracy = np.mean(fold_accuracies)

        final_model = model_creator(embedding_matrix, max_len)
        final_model.fit(X_train_dl, y_train_dl, epochs=10, batch_size=32, verbose=0)
        y_pred_probs = final_model.predict(X_test_dl)
        y_pred = np.argmax(y_pred_probs, axis=1)
        y_true = np.argmax(y_test_dl, axis=1)

        accuracy = accuracy_score(y_true, y_pred)
        report = classification_report(y_true, y_pred, target_names=sentiment_mapping.keys(), zero_division=0)
        key = f"{model_name}_Word2Vec_DL"
        results[key] = {'model': final_model, 'accuracy': accuracy, 'report': report, 'avg_cv_accuracy': avg_cv_accuracy, 'type': 'dl'}

    # --- Loop Pelatihan Model Machine Learning (Word2Vec-Avg, BoW, TF-IDF) ---
    for feature_name, (X_train, X_test) in ml_feature_sets.items():
        for model_name, model_instance in models_to_evaluate_ml.items():
            print(f"\nMelatih model {model_name} dengan fitur {feature_name}...")
            kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
            fold_accuracies = []

            for train, val in kfold.split(X_train, y_train_ml_np):
                model = clone(model_instance)
                model.fit(X_train[train], y_train_ml_np[train])
                acc = model.score(X_train[val], y_train_ml_np[val])
                fold_accuracies.append(acc)

            avg_cv_accuracy = np.mean(fold_accuracies)

            final_model = clone(model_instance)
            final_model.fit(X_train, y_train_ml_np)
            y_pred = final_model.predict(X_test)

            accuracy = accuracy_score(y_test_ml_np, y_pred)
            report = classification_report(y_test_ml_np, y_pred, target_names=sentiment_mapping.keys(), zero_division=0)
            key = f"{model_name}_{feature_name}"

            if feature_name == 'Word2Vec_Avg':
                model_type = 'ml_w2v'
            elif feature_name == 'BoW':
                model_type = 'ml_bow'
            else: # TFIDF
                model_type = 'ml_tfidf'

            results[key] = {'model': final_model, 'accuracy': accuracy, 'report': report, 'avg_cv_accuracy': avg_cv_accuracy, 'type': model_type}

    # --- F. Menampilkan Hasil dan Memilih Model Terbaik ---
    print("\n--- Hasil Evaluasi Akhir ---")
    best_model_key = None
    best_accuracy = 0
    for key, value in results.items():
        print(f"Kombinasi: {key} (Tipe: {value['type']})")
        print(f"  Akurasi pada Test Set: {value['accuracy']:.4f}")
        print(f"  Rata-rata Akurasi CV: {value['avg_cv_accuracy']:.4f}")
        print("  Laporan Klasifikasi:\n", value['report'])

        if value['accuracy'] > best_accuracy:
            best_accuracy = value['accuracy']
            best_model_key = key

    print(f"\nModel terbaik adalah: {best_model_key} dengan akurasi test set {best_accuracy:.4f}")

    # --- G. Implementasi Model Terbaik pada Data Sisa (70%) ---
    if best_model_key is None:
        print("\nTidak ada model yang berhasil dilatih. Implementasi dibatalkan.")
    else:
        best_model_data = results[best_model_key]
        best_model = best_model_data['model']
        best_model_type = best_model_data['type']

        print(f"\nMelakukan prediksi pada 70% data sisa menggunakan model: {best_model_key}")

        if best_model_type == 'dl':
            unannotated_sequences = tokenizer.texts_to_sequences(df_unannotated['cleaned_text_nouns'])
            X_unannotated_feed = pad_sequences(unannotated_sequences, maxlen=max_len, padding='post', truncating='post')
            predictions_probs = best_model.predict(X_unannotated_feed)
            predictions = np.argmax(predictions_probs, axis=1)

        elif best_model_type == 'ml_w2v':
            X_unannotated_feed = create_document_vectors(df_unannotated['cleaned_tokens_nouns'], w2v_model, w2v_model.vector_size)
            predictions = best_model.predict(X_unannotated_feed)

        elif best_model_type == 'ml_bow':
            X_unannotated_feed = bow_vectorizer.transform(df_unannotated['cleaned_text_nouns'])
            predictions = best_model.predict(X_unannotated_feed)

        elif best_model_type == 'ml_tfidf':
            X_unannotated_feed = tfidf_vectorizer.transform(df_unannotated['cleaned_text_nouns'])
            predictions = best_model.predict(X_unannotated_feed)

        inverse_sentiment_mapping = {v: k for k, v in sentiment_mapping.items()}
        df_unannotated[f'predicted_{target_aspect}'] = [inverse_sentiment_mapping.get(p, 'neutral') for p in predictions]

        output_filename = f'hasil_prediksi_{target_aspect}.csv'
        df_unannotated.to_csv(output_filename, index=False)

        print(f"Prediksi sentimen telah selesai.")
        print(f"Hasil disimpan di file: {output_filename}")

    print("\n--- Analisis Selesai ---")

NLTK stopwords (English) sudah ada.
NLTK punkt_tab sudah ada.
NLTK POS Tagger (averaged_perceptron_tagger) sudah ada.
NLTK POS Tagger (averaged_perceptron_tagger_eng) sudah ada.
Persiapan NLTK selesai.

[INFO] Mempersiapkan Sastrawi Stemmer dan Stopword Remover...
Sastrawi dan stopwords gabungan siap.
INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Jumlah data beranotasi: 3000
Jumlah data belum beranotasi: 7747
[INFO] Menerapkan prapemrosesan (Stemming, POS Tag, Filter Nouns, Stopwords)...
Memproses df_annotated (mungkin butuh waktu)...


Memproses df_unannotated (mungkin butuh waktu)...



--- Menyiapkan Model Embedding (Word2Vec) ---
Model Word2Vec selesai dilatih.

--- Mempersiapkan Fitur Set Machine Learning ---
Fitur Word2Vec (Avg) siap.
Fitur Bag of Words (BoW) siap.
Fitur TF-IDF siap.

--- Memulai Pelatihan dan Evaluasi Model ---

Melatih model CNN dengan embedding Word2Vev...
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step

Melatih model LSTM dengan embedding Word2Vev...
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step

Melatih model RNN dengan embedding Word2Vev...
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step

Melatih model LogisticRegression dengan fitur Word2Vec_Avg...

Melatih model DecisionTree dengan fitur Word2Vec_Avg...

Melatih model RandomForest dengan fitur Word2Vec_Avg...

Melatih model XGBoost dengan fitur Word2Vec_Avg...

Melatih model LogisticRegression dengan fitur BoW...

Melatih model DecisionTree dengan fitur BoW...

Melatih model RandomForest dengan fitur BoW...

Melatih model XGBoost dengan fitur BoW...

Melatih model LogisticRegression dengan fitur TFIDF...

Mela

In [27]:
# -*- coding: utf-8 -*-
"""
Skrip Python KomprehensIF untuk Analisis Sentimen Berbasis Aspek (Versi Diperluas)
Tujuan: Menganalisis sentimen wisatawan terhadap event MotoGP di Sirkuit Mandalika.
Metodologi:
1.  Prapemrosesan Data: Tokenisasi, Stemming (Sastrawi), POS Tagging (NLTK-EN),
    Filter Kata Benda (Nouns), Stopword Removal (Sastrawi-ID + NLTK-EN).
2.  Ekstraksi Fitur DL: Word2Vec (dilatih pada korpus).
3.  Ekstraksi Fitur ML: Word2Vec (Rata-rata), Bag of Words (BoW), dan TF-IDF.
4.  Pelatihan Model Deep Learning: CNN, LSTM, RNN (menggunakan Word2Vec).
5.  Pelatihan Model Machine Learning: Logistic Regression, Decision Tree, Random Forest, XGBoost
    (menggunakan W2V-Avg, BoW, dan TF-IDF).
6.  Validasi Silang (3-Fold Cross-Validation) untuk semua model.
7.  Implementasi model terbaik untuk memprediksi sentimen pada data sisa (unlabeled).

PENTING: Skrip ini berasumsi dua pandas DataFrames bernama 'df_annotated'
dan 'df_unannotated' sudah ada di environment Anda.
"""

# Import library yang dibutuhkan
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover import StopWordRemoverFactory
from pandarallel import pandarallel
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, LSTM, SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import warnings

# Mute warnings dari sklearn dan xgboost
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# --- 1. PERSIAPAN AWAL & DOWNLOAD DEPENDENSI NLTK ---
try:
    stopwords.words('english') # Hanya NLTK English stopwords yang kita perlukan
    print("NLTK stopwords (English) sudah ada.")
except LookupError:
    print("Mengunduh NLTK stopwords (English)...")
    nltk.download('stopwords')

try:
    nltk.data.find('tokenizers/punkt_tab')
    print("NLTK punkt_tab sudah ada.")
except LookupError:
    print("Mengunduh 'punkt_tab' yang hilang...")
    nltk.download('punkt_tab')

try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
    print("NLTK POS Tagger (averaged_perceptron_tagger) sudah ada.")
except LookupError:
    print("Mengunduh NLTK POS Tagger...")
    nltk.download('averaged_perceptron_tagger')

try:
    nltk.data.find('taggers/averaged_perceptron_tagger_eng')
    print("NLTK POS Tagger (averaged_perceptron_tagger_eng) sudah ada.")
except LookupError:
    print("Mengunduh NLTK POS Tagger (sub-modul 'eng')...")
    nltk.download('averaged_perceptron_tagger_eng')

print("Persiapan NLTK selesai.")

# --- 2. FUNGSI PRAPEMROSESAN DATA (DENGAN STEMMING & POS TAGGING) ---
def preprocess_text_with_pos(text, stemmer, combined_stopwords):
    """
    Fungsi untuk membersihkan teks, melakukan stemming (Sastrawi),
    POS tagging (NLTK-EN), memfilter kata benda (nouns),
    dan menghapus stopwords (Sastrawi-ID + NLTK-EN).

    Mengembalikan:
    1. List token kata benda yang sudah bersih.
    2. String gabungan dari token kata benda yang sudah bersih.
    """
    if not isinstance(text, str):
        return [], ""

    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)

    # 1. Tokenisasi
    tokens = word_tokenize(text)

    # 2. Stemming (Sastrawi)
    # Diterapkan sebelum POS Tagging untuk menormalkan kata-kata Bahasa Indonesia.
    # Stemmer Sastrawi cukup pintar untuk mengabaikan kata-kata Bahasa Inggris.
    stemmed_tokens = [stemmer.stem(word) for word in tokens]

    # 3. POS Tagging (NLTK - English)
    # Tagger ini akan bekerja pada token yang sudah di-stem.
    pos_tags = nltk.pos_tag(stemmed_tokens)

    # 4. Filter Kata Benda (Nouns)
    # NN (noun, singular), NNS (noun, plural), NNP (proper noun, singular), NNPS (proper noun, plural)
    noun_tokens = [word for word, tag in pos_tags if tag in ('NN', 'NNS', 'NNP', 'NNPS')]

    # 5. Hapus Stopwords
    # Menggunakan daftar gabungan yang sudah disiapkan sebelumnya
    filtered_nouns = [word for word in noun_tokens if word not in combined_stopwords and len(word) > 2]

    return filtered_nouns, " ".join(filtered_nouns)


# --- 3. EKSTRAKSI FITUR (WORD EMBEDDING) ---
def train_word2vec(corpus_tokens, vector_size=100, window=5, min_count=2):
    """Melatih model Word2Vec dari korpus teks multibahasa (list of lists)."""
    model = Word2Vec(corpus_tokens, vector_size=vector_size, window=window, min_count=min_count, workers=4)
    return model

def create_embedding_matrix_from_word2vec(word2vec_model, word_index):
    """Membuat embedding matrix dari model Word2Vec yang sudah dilatih (untuk Keras)."""
    embedding_dim = word2vec_model.vector_size
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in word2vec_model.wv:
            embedding_matrix[i] = word2vec_model.wv[word]
    return embedding_matrix

def create_document_vectors(corpus_tokenized, model, embedding_dim):
    """
    Membuat vektor dokumen rata-rata untuk model ML klasik.
    """
    vectors = []
    for doc_tokens in corpus_tokenized:
        doc_vec = np.zeros(embedding_dim)
        count = 0
        for word in doc_tokens:
            if word in model.wv:
                doc_vec += model.wv[word]
                count += 1
        if count > 0:
            doc_vec /= count  # Rata-rata vektor
        vectors.append(doc_vec)
    return np.array(vectors)

# --- 4. PEMBUATAN MODEL ---
# (Fungsi create_cnn_model, create_lstm_model, create_rnn_model tetap sama)
def create_cnn_model(embedding_matrix, max_len):
    model = Sequential([
        Embedding(input_dim=embedding_matrix.shape[0],
                  output_dim=embedding_matrix.shape[1],
                  weights=[embedding_matrix],
                  input_length=max_len,
                  trainable=False),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_lstm_model(embedding_matrix, max_len):
    model = Sequential([
        Embedding(input_dim=embedding_matrix.shape[0],
                  output_dim=embedding_matrix.shape[1],
                  weights=[embedding_matrix],
                  input_length=max_len,
                  trainable=False),
        LSTM(64, dropout=0.2, recurrent_dropout=0.2),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_rnn_model(embedding_matrix, max_len):
    model = Sequential([
        Embedding(input_dim=embedding_matrix.shape[0],
                  output_dim=embedding_matrix.shape[1],
                  weights=[embedding_matrix],
                  input_length=max_len,
                  trainable=False),
        SimpleRNN(64),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# --- 5. PROSES UTAMA (MAIN WORKFLOW) ---
if __name__ == '__main__':

    # # --- TAMBAHAN BARU: Inisialisasi Sastrawi & Stopwords Gabungan ---
    # print("\n[INFO] Mempersiapkan Sastrawi Stemmer dan Stopword Remover...")
    # # 1. Stemmer
    # stemmer_factory = StemmerFactory()
    # stemmer = stemmer_factory.create_stemmer()

    # # 2. Stopwords
    # stopword_factory = StopWordRemoverFactory.StopWordRemoverFactory()
    # sastrawi_stopwords = set(stopword_factory.get_stop_words())
    # nltk_english_stopwords = set(stopwords.words('english'))
    # COMBINED_STOPWORDS = sastrawi_stopwords.union(nltk_english_stopwords)
    # print("Sastrawi dan stopwords gabungan siap.")
    # # --- AKHIR TAMBAHAN BARU ---

    # # --- TAMBAHAN BARU: Inisialisasi Pandarallel ---
    # # Menggunakan semua inti CPU yang tersedia
    # pandarallel.initialize(progress_bar=True)

    # --- A. Muat dan Proses Data ---
    if 'df_annotated' not in locals() or 'df_unannotated' not in locals():
        print("="*50)
        print("ERROR: Pastikan 'df_annotated' dan 'df_unannotated' sudah dimuat.")
        print("Contoh: df_annotated = pd.read_csv('data_annotated_30persen.csv')")
        print("="*50)
        raise NameError("'df_annotated' or 'df_unannotated' not defined.")

    print(f"Jumlah data beranotasi: {len(df_annotated)}")
    print(f"Jumlah data belum beranotasi: {len(df_unannotated)}")


    # # --- PERUBAHAN: Menerapkan prapemrosesan baru dengan Sastrawi ---
    # print("[INFO] Menerapkan prapemrosesan (Stemming, POS Tag, Filter Nouns, Stopwords)...")
    # print("Memproses df_annotated (mungkin butuh waktu)...")
    # # Terapkan pada data beranotasi (MENGGUNAKAN parallel_apply)
    # df_annotated[['cleaned_tokens_nouns', 'cleaned_text_nouns']] = \
    #     df_annotated['text'].parallel_apply(lambda x: pd.Series(preprocess_text_with_pos(x, stemmer, COMBINED_STOPWORDS)))

    # print("Memproses df_unannotated (mungkin butuh waktu)...")
    # # Terapkan pada data belum beranotasi (MENGGUNAKAN parallel_apply)
    # df_unannotated[['cleaned_tokens_nouns', 'cleaned_text_nouns']] = \
    #     df_unannotated['text'].parallel_apply(lambda x: pd.Series(preprocess_text_with_pos(x, stemmer, COMBINED_STOPWORDS)))

    # Gabungkan semua teks (string) yang sudah bersih untuk vectorizer ML
    all_corpus_text_nouns = pd.concat([df_annotated['cleaned_text_nouns'], df_unannotated['cleaned_text_nouns']], ignore_index=True)
    # Gabungkan semua token (list) yang sudah bersih untuk Word2Vec
    all_corpus_tokens_nouns = pd.concat([df_annotated['cleaned_tokens_nouns'], df_unannotated['cleaned_tokens_nouns']], ignore_index=True).tolist()
    # --- AKHIR PERUBAHAN ---

    target_aspect = 'circuit_sentiment'

    sentiment_mapping = {'positive': 0, 'neutral': 1, 'negative': 2}
    df_annotated[target_aspect] = df_annotated[target_aspect].map(sentiment_mapping)
    df_annotated = df_annotated.dropna(subset=[target_aspect])

    # --- B. Persiapan Data (Label dan Fitur) ---

    # 1. Label
    y_categorical = tf.keras.utils.to_categorical(df_annotated[target_aspect].astype(int), num_classes=3)
    y_1d = df_annotated[target_aspect].astype(int) # Untuk model ML

    # 2. Fitur untuk Deep Learning (Tokenized & Padded Sequences)
    tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
    tokenizer.fit_on_texts(all_corpus_text_nouns)
    word_index = tokenizer.word_index
    sequences = tokenizer.texts_to_sequences(df_annotated['cleaned_text_nouns'])
    max_len = 100
    X_padded = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

    # Split data DL
    X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(
        X_padded, y_categorical, test_size=0.2, random_state=42, stratify=y_1d
    )

    # 3. Fitur untuk Machine Learning
    X_train_text_ml, X_test_text_ml, y_train_ml, y_test_ml = train_test_split(
        df_annotated, y_1d, test_size=0.2, random_state=42, stratify=y_1d
    )

    # --- C. Persiapan Model Embedding (Word2Vec) ---
    print("\n--- Menyiapkan Model Embedding (Word2Vec) ---")
    w2v_model = train_word2vec(all_corpus_tokens_nouns)
    print("Model Word2Vec selesai dilatih.")

    # --- D. Persiapan Fitur Set untuk ML ---
    print("\n--- Mempersiapkan Fitur Set Machine Learning ---")
    ml_feature_sets = {}

    # Fitur 1: Word2Vec (Rata-rata)
    embedding_dim = w2v_model.vector_size
    X_train_ml_w2v = create_document_vectors(X_train_text_ml['cleaned_tokens_nouns'], w2v_model, embedding_dim)
    X_test_ml_w2v = create_document_vectors(X_test_text_ml['cleaned_tokens_nouns'], w2v_model, embedding_dim)
    ml_feature_sets['Word2Vec_Avg'] = (X_train_ml_w2v, X_test_ml_w2v)
    print("Fitur Word2Vec (Avg) siap.")

    # Fitur 2: Bag of Words (BoW)
    bow_vectorizer = CountVectorizer(max_features=5000)
    X_train_ml_bow = bow_vectorizer.fit_transform(X_train_text_ml['cleaned_text_nouns'])
    X_test_ml_bow = bow_vectorizer.transform(X_test_text_ml['cleaned_text_nouns'])
    ml_feature_sets['BoW'] = (X_train_ml_bow, X_test_ml_bow)
    print("Fitur Bag of Words (BoW) siap.")

    # Fitur 3: TF-IDF
    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    X_train_ml_tfidf = tfidf_vectorizer.fit_transform(X_train_text_ml['cleaned_text_nouns'])
    X_test_ml_tfidf = tfidf_vectorizer.transform(X_test_text_ml['cleaned_text_nouns'])
    ml_feature_sets['TFIDF'] = (X_train_ml_tfidf, X_test_ml_tfidf)
    print("Fitur TF-IDF siap.")

    y_train_ml_np = y_train_ml.to_numpy()
    y_test_ml_np = y_test_ml.to_numpy()

    # --- E. Training & Evaluasi Model ---
    print("\n--- Memulai Pelatihan dan Evaluasi Model ---")

    models_to_evaluate_dl = {'CNN': create_cnn_model, 'LSTM': create_lstm_model, 'RNN': create_rnn_model}
    models_to_evaluate_ml = {
        'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
        'DecisionTree': DecisionTreeClassifier(random_state=42),
        'RandomForest': RandomForestClassifier(random_state=42),
        'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
    }

    results = {}

    # --- Loop Pelatihan Model Deep Learning (Hanya Word2Vec) ---
    embedding_matrix = create_embedding_matrix_from_word2vec(w2v_model, word_index)

    for model_name, model_creator in models_to_evaluate_dl.items():
        print(f"\nMelatih model {model_name} dengan embedding Word2Vev...")
        kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        fold_accuracies = []

        for train, val in kfold.split(X_train_dl, np.argmax(y_train_dl, axis=1)):
            model = model_creator(embedding_matrix, max_len)
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
            model.fit(X_train_dl[train], y_train_dl[train],
                      epochs=20, batch_size=32,
                      validation_data=(X_train_dl[val], y_train_dl[val]),
                      callbacks=[early_stopping], verbose=0)
            _, acc = model.evaluate(X_train_dl[val], y_train_dl[val], verbose=0)
            fold_accuracies.append(acc)

        avg_cv_accuracy = np.mean(fold_accuracies)

        final_model = model_creator(embedding_matrix, max_len)
        final_model.fit(X_train_dl, y_train_dl, epochs=10, batch_size=32, verbose=0)
        y_pred_probs = final_model.predict(X_test_dl)
        y_pred = np.argmax(y_pred_probs, axis=1)
        y_true = np.argmax(y_test_dl, axis=1)

        accuracy = accuracy_score(y_true, y_pred)
        report = classification_report(y_true, y_pred, target_names=sentiment_mapping.keys(), zero_division=0)
        key = f"{model_name}_Word2Vec_DL"
        results[key] = {'model': final_model, 'accuracy': accuracy, 'report': report, 'avg_cv_accuracy': avg_cv_accuracy, 'type': 'dl'}

    # --- Loop Pelatihan Model Machine Learning (Word2Vec-Avg, BoW, TF-IDF) ---
    for feature_name, (X_train, X_test) in ml_feature_sets.items():
        for model_name, model_instance in models_to_evaluate_ml.items():
            print(f"\nMelatih model {model_name} dengan fitur {feature_name}...")
            kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
            fold_accuracies = []

            for train, val in kfold.split(X_train, y_train_ml_np):
                model = clone(model_instance)
                model.fit(X_train[train], y_train_ml_np[train])
                acc = model.score(X_train[val], y_train_ml_np[val])
                fold_accuracies.append(acc)

            avg_cv_accuracy = np.mean(fold_accuracies)

            final_model = clone(model_instance)
            final_model.fit(X_train, y_train_ml_np)
            y_pred = final_model.predict(X_test)

            accuracy = accuracy_score(y_test_ml_np, y_pred)
            report = classification_report(y_test_ml_np, y_pred, target_names=sentiment_mapping.keys(), zero_division=0)
            key = f"{model_name}_{feature_name}"

            if feature_name == 'Word2Vec_Avg':
                model_type = 'ml_w2v'
            elif feature_name == 'BoW':
                model_type = 'ml_bow'
            else: # TFIDF
                model_type = 'ml_tfidf'

            results[key] = {'model': final_model, 'accuracy': accuracy, 'report': report, 'avg_cv_accuracy': avg_cv_accuracy, 'type': model_type}

    # --- F. Menampilkan Hasil dan Memilih Model Terbaik ---
    print("\n--- Hasil Evaluasi Akhir ---")
    best_model_key = None
    best_accuracy = 0
    for key, value in results.items():
        print(f"Kombinasi: {key} (Tipe: {value['type']})")
        print(f"  Akurasi pada Test Set: {value['accuracy']:.4f}")
        print(f"  Rata-rata Akurasi CV: {value['avg_cv_accuracy']:.4f}")
        print("  Laporan Klasifikasi:\n", value['report'])

        if value['accuracy'] > best_accuracy:
            best_accuracy = value['accuracy']
            best_model_key = key

    print(f"\nModel terbaik adalah: {best_model_key} dengan akurasi test set {best_accuracy:.4f}")

    # --- G. Implementasi Model Terbaik pada Data Sisa (70%) ---
    if best_model_key is None:
        print("\nTidak ada model yang berhasil dilatih. Implementasi dibatalkan.")
    else:
        best_model_data = results[best_model_key]
        best_model = best_model_data['model']
        best_model_type = best_model_data['type']

        print(f"\nMelakukan prediksi pada 70% data sisa menggunakan model: {best_model_key}")

        if best_model_type == 'dl':
            unannotated_sequences = tokenizer.texts_to_sequences(df_unannotated['cleaned_text_nouns'])
            X_unannotated_feed = pad_sequences(unannotated_sequences, maxlen=max_len, padding='post', truncating='post')
            predictions_probs = best_model.predict(X_unannotated_feed)
            predictions = np.argmax(predictions_probs, axis=1)

        elif best_model_type == 'ml_w2v':
            X_unannotated_feed = create_document_vectors(df_unannotated['cleaned_tokens_nouns'], w2v_model, w2v_model.vector_size)
            predictions = best_model.predict(X_unannotated_feed)

        elif best_model_type == 'ml_bow':
            X_unannotated_feed = bow_vectorizer.transform(df_unannotated['cleaned_text_nouns'])
            predictions = best_model.predict(X_unannotated_feed)

        elif best_model_type == 'ml_tfidf':
            X_unannotated_feed = tfidf_vectorizer.transform(df_unannotated['cleaned_text_nouns'])
            predictions = best_model.predict(X_unannotated_feed)

        inverse_sentiment_mapping = {v: k for k, v in sentiment_mapping.items()}
        df_unannotated[f'predicted_{target_aspect}'] = [inverse_sentiment_mapping.get(p, 'neutral') for p in predictions]

        output_filename = f'hasil_prediksi_{target_aspect}.csv'
        df_unannotated.to_csv(output_filename, index=False)

        print(f"Prediksi sentimen telah selesai.")
        print(f"Hasil disimpan di file: {output_filename}")

    print("\n--- Analisis Selesai ---")

NLTK stopwords (English) sudah ada.
NLTK punkt_tab sudah ada.
NLTK POS Tagger (averaged_perceptron_tagger) sudah ada.
NLTK POS Tagger (averaged_perceptron_tagger_eng) sudah ada.
Persiapan NLTK selesai.
Jumlah data beranotasi: 3000
Jumlah data belum beranotasi: 7747

--- Menyiapkan Model Embedding (Word2Vec) ---
Model Word2Vec selesai dilatih.

--- Mempersiapkan Fitur Set Machine Learning ---
Fitur Word2Vec (Avg) siap.
Fitur Bag of Words (BoW) siap.
Fitur TF-IDF siap.

--- Memulai Pelatihan dan Evaluasi Model ---

Melatih model CNN dengan embedding Word2Vev...
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step

Melatih model LSTM dengan embedding Word2Vev...
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step

Melatih model RNN dengan embedding Word2Vev...
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step

Melatih model LogisticRegression dengan fitur Word2Vec_Avg...

Melatih model DecisionTree dengan fitur Word2Vec_Avg...

Melatih model RandomForest dengan fitur Word2Vec_Avg...

Melatih model XGBoost dengan fitur Word

In [22]:
# -*- coding: utf-8 -*-
"""
Skrip Python KomprehensIF untuk Analisis Sentimen Berbasis Aspek (Versi Diperluas)
Tujuan: Menganalisis sentimen wisatawan terhadap event MotoGP di Sirkuit Mandalika.
Metodologi:
1.  Prapemrosesan Data: Tokenisasi, POS Tagging, Filter Kata Benda (Nouns), Stopword Removal.
2.  Ekstraksi Fitur DL: Word2Vec (dilatih pada korpus).
3.  Ekstraksi Fitur ML: Word2Vec (Rata-rata), Bag of Words (BoW), dan TF-IDF.
4.  Pelatihan Model Deep Learning: CNN, LSTM, RNN (menggunakan Word2Vec).
5.  Pelatihan Model Machine Learning: Logistic Regression, Decision Tree, Random Forest, XGBoost
    (menggunakan W2V-Avg, BoW, dan TF-IDF).
6.  Validasi Silang (3-Fold Cross-Validation) untuk semua model.
7.  Implementasi model terbaik untuk memprediksi sentimen pada data sisa (unlabeled).

PENTING: Skrip ini berasumsi dua pandas DataFrames bernama 'df_annotated'
dan 'df_unannotated' sudah ada di environment Anda.
"""

# Import library yang dibutuhkan
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.base import clone
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
# --- TAMBAHAN ---
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# --- AKHIR TAMBAHAN ---
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, LSTM, SimpleRNN, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import warnings

# Mute warnings dari sklearn dan xgboost
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# --- 1. PERSIAPAN AWAL & DOWNLOAD DEPENDENSI ---
try:
    stopwords.words('indonesian')
    stopwords.words('english')
    print("NLTK stopwords sudah ada.")
except LookupError:
    print("Mengunduh NLTK stopwords untuk Bahasa Indonesia dan Inggris...")
    nltk.download('stopwords')

try:
    nltk.data.find('tokenizers/punkt_tab')
    print("NLTK punkt_tab sudah ada.")
except LookupError:
    print("Mengunduh 'punkt_tab' yang hilang...")
    nltk.download('punkt_tab')

# --- TAMBAHAN: Download POS Tagger ---
try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
    print("NLTK POS Tagger (averaged_perceptron_tagger) sudah ada.")
except LookupError:
    print("Mengunduh NLTK POS Tagger...")
    nltk.download('averaged_perceptron_tagger')
# --- AKHIR TAMBAHAN ---

# --- TAMBAHAN BARU: Download sub-modul 'eng' tagger ---
# nltk.pos_tag secara internal memanggil ini
try:
    nltk.data.find('taggers/averaged_perceptron_tagger_eng')
    print("NLTK POS Tagger (averaged_perceptron_tagger_eng) sudah ada.")
except LookupError:
    print("Mengunduh NLTK POS Tagger (sub-modul 'eng')...")
    nltk.download('averaged_perceptron_tagger_eng')
# --- AKHIR TAMBAHAN BARU ---
# --- AKHIR TAMBAHAN ---

print("Persiapan awal selesai.")

# --- 2. FUNGSI PRAPEMROSESAN DATA (DENGAN POS TAGGING) ---
def preprocess_text_with_pos(text):
    """
    Fungsi untuk membersihkan teks, melakukan POS tagging, memfilter kata benda (nouns),
    dan menghapus stopwords.

    PERINGATAN: NLTK POS Tagger (nltk.pos_tag) dilatih pada bahasa Inggris.
    Performanya mungkin tidak optimal untuk kata-kata Bahasa Indonesia,
    meskipun seringkali ia akan menandai kata yang tidak dikenal sebagai 'NN' (noun).

    Mengembalikan:
    1. List token kata benda yang sudah bersih.
    2. String gabungan dari token kata benda yang sudah bersih.
    """
    if not isinstance(text, str):
        return [], ""

    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^a-z\s]', '', text)

    # 1. Tokenisasi
    tokens = word_tokenize(text)

    # 2. POS Tagging
    pos_tags = nltk.pos_tag(tokens)

    # 3. Filter Kata Benda (Nouns)
    # NN (noun, singular), NNS (noun, plural), NNP (proper noun, singular), NNPS (proper noun, plural)
    noun_tokens = [word for word, tag in pos_tags if tag in ('NN', 'NNS', 'NNP', 'NNPS')]

    # 4. Hapus Stopwords
    stop_words_id = set(stopwords.words('indonesian'))
    stop_words_en = set(stopwords.words('english'))
    combined_stop_words = stop_words_id.union(stop_words_en)

    filtered_nouns = [word for word in noun_tokens if word not in combined_stop_words and len(word) > 2]

    return filtered_nouns, " ".join(filtered_nouns)


# --- 3. EKSTRAKSI FITUR (WORD EMBEDDING) ---
def train_word2vec(corpus_tokens, vector_size=100, window=5, min_count=2):
    """Melatih model Word2Vec dari korpus teks multibahasa (list of lists)."""
    # corpus_tokens sekarang adalah list of lists (bukan list of strings)
    model = Word2Vec(corpus_tokens, vector_size=vector_size, window=window, min_count=min_count, workers=4)
    return model

def create_embedding_matrix_from_word2vec(word2vec_model, word_index):
    """Membuat embedding matrix dari model Word2Vec yang sudah dilatih (untuk Keras)."""
    embedding_dim = word2vec_model.vector_size
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in word2vec_model.wv:
            embedding_matrix[i] = word2vec_model.wv[word]
    return embedding_matrix

def create_document_vectors(corpus_tokenized, model, embedding_dim):
    """
    Membuat vektor dokumen rata-rata untuk model ML klasik.
    """
    vectors = []
    for doc_tokens in corpus_tokenized:
        doc_vec = np.zeros(embedding_dim)
        count = 0
        for word in doc_tokens:
            if word in model.wv:
                doc_vec += model.wv[word]
                count += 1
        if count > 0:
            doc_vec /= count  # Rata-rata vektor
        vectors.append(doc_vec)
    return np.array(vectors)

# --- 4. PEMBUATAN MODEL ---
# (Fungsi create_cnn_model, create_lstm_model, create_rnn_model tetap sama)
def create_cnn_model(embedding_matrix, max_len):
    model = Sequential([
        Embedding(input_dim=embedding_matrix.shape[0],
                  output_dim=embedding_matrix.shape[1],
                  weights=[embedding_matrix],
                  input_length=max_len,
                  trainable=False),
        Conv1D(128, 5, activation='relu'),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_lstm_model(embedding_matrix, max_len):
    model = Sequential([
        Embedding(input_dim=embedding_matrix.shape[0],
                  output_dim=embedding_matrix.shape[1],
                  weights=[embedding_matrix],
                  input_length=max_len,
                  trainable=False),
        LSTM(64, dropout=0.2, recurrent_dropout=0.2),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_rnn_model(embedding_matrix, max_len):
    model = Sequential([
        Embedding(input_dim=embedding_matrix.shape[0],
                  output_dim=embedding_matrix.shape[1],
                  weights=[embedding_matrix],
                  input_length=max_len,
                  trainable=False),
        SimpleRNN(64),
        Dense(32, activation='relu'),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# --- 5. PROSES UTAMA (MAIN WORKFLOW) ---
if __name__ == '__main__':

    # --- A. Muat dan Proses Data ---
    if 'df_annotated' not in locals() or 'df_unannotated' not in locals():
        print("="*50)
        print("ERROR: Pastikan 'df_annotated' dan 'df_unannotated' sudah dimuat.")
        print("Contoh: df_annotated = pd.read_csv('data_annotated_30persen.csv')")
        print("="*50)
        raise NameError("'df_annotated' or 'df_unannotated' not defined.")

    print(f"Jumlah data beranotasi: {len(df_annotated)}")
    print(f"Jumlah data belum beranotasi: {len(df_unannotated)}")

    # --- PERUBAHAN: Menerapkan prapemrosesan baru ---
    print("Menerapkan prapemrosesan (Tokenize, POS Tag, Filter Nouns, Stopwords)...")
    # Terapkan pada data beranotasi
    df_annotated[['cleaned_tokens_nouns', 'cleaned_text_nouns']] = \
        df_annotated['text'].apply(lambda x: pd.Series(preprocess_text_with_pos(x)))

    # Terapkan pada data belum beranotasi
    df_unannotated[['cleaned_tokens_nouns', 'cleaned_text_nouns']] = \
        df_unannotated['text'].apply(lambda x: pd.Series(preprocess_text_with_pos(x)))

    # Gabungkan semua teks (string) yang sudah bersih untuk vectorizer ML
    all_corpus_text_nouns = pd.concat([df_annotated['cleaned_text_nouns'], df_unannotated['cleaned_text_nouns']], ignore_index=True)
    # Gabungkan semua token (list) yang sudah bersih untuk Word2Vec
    all_corpus_tokens_nouns = pd.concat([df_annotated['cleaned_tokens_nouns'], df_unannotated['cleaned_tokens_nouns']], ignore_index=True).tolist()
    # --- AKHIR PERUBAHAN ---

    target_aspect = 'circuit_sentiment'

    sentiment_mapping = {'positive': 0, 'neutral': 1, 'negative': 2}
    df_annotated[target_aspect] = df_annotated[target_aspect].map(sentiment_mapping)
    df_annotated = df_annotated.dropna(subset=[target_aspect])

    # --- B. Persiapan Data (Label dan Fitur) ---

    # 1. Label
    y_categorical = tf.keras.utils.to_categorical(df_annotated[target_aspect].astype(int), num_classes=3)
    y_1d = df_annotated[target_aspect].astype(int) # Untuk model ML

    # 2. Fitur untuk Deep Learning (Tokenized & Padded Sequences)
    tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
    # Fit pada semua teks (string) kata benda
    tokenizer.fit_on_texts(all_corpus_text_nouns)
    word_index = tokenizer.word_index
    # Ubah teks beranotasi (string) menjadi sequence
    sequences = tokenizer.texts_to_sequences(df_annotated['cleaned_text_nouns'])
    max_len = 100
    X_padded = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

    # Split data DL
    X_train_dl, X_test_dl, y_train_dl, y_test_dl = train_test_split(
        X_padded, y_categorical, test_size=0.2, random_state=42, stratify=y_1d
    )

    # 3. Fitur untuk Machine Learning
    #    Kita akan split data teks terlebih dahulu, baru terapkan vectorizer
    #    untuk mencegah data leakage dari test set.
    X_train_text_ml, X_test_text_ml, y_train_ml, y_test_ml = train_test_split(
        df_annotated, y_1d, test_size=0.2, random_state=42, stratify=y_1d
    )

    # --- C. Persiapan Model Embedding (Word2Vec) ---
    print("\n--- Menyiapkan Model Embedding (Word2Vec) ---")
    # Latih Word2Vec pada semua token (list of lists)
    w2v_model = train_word2vec(all_corpus_tokens_nouns)
    print("Model Word2Vec selesai dilatih.")

    # --- D. Persiapan Fitur Set untuk ML ---
    print("\n--- Mempersiapkan Fitur Set Machine Learning ---")
    ml_feature_sets = {}

    # Fitur 1: Word2Vec (Rata-rata)
    embedding_dim = w2v_model.vector_size
    X_train_ml_w2v = create_document_vectors(X_train_text_ml['cleaned_tokens_nouns'], w2v_model, embedding_dim)
    X_test_ml_w2v = create_document_vectors(X_test_text_ml['cleaned_tokens_nouns'], w2v_model, embedding_dim)
    ml_feature_sets['Word2Vec_Avg'] = (X_train_ml_w2v, X_test_ml_w2v)
    print("Fitur Word2Vec (Avg) siap.")

    # Fitur 2: Bag of Words (BoW)
    bow_vectorizer = CountVectorizer(max_features=5000)
    X_train_ml_bow = bow_vectorizer.fit_transform(X_train_text_ml['cleaned_text_nouns'])
    X_test_ml_bow = bow_vectorizer.transform(X_test_text_ml['cleaned_text_nouns'])
    ml_feature_sets['BoW'] = (X_train_ml_bow, X_test_ml_bow)
    print("Fitur Bag of Words (BoW) siap.")

    # Fitur 3: TF-IDF
    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    X_train_ml_tfidf = tfidf_vectorizer.fit_transform(X_train_text_ml['cleaned_text_nouns'])
    X_test_ml_tfidf = tfidf_vectorizer.transform(X_test_text_ml['cleaned_text_nouns'])
    ml_feature_sets['TFIDF'] = (X_train_ml_tfidf, X_test_ml_tfidf)
    print("Fitur TF-IDF siap.")

    # Konversi label ML ke numpy (PENTING untuk KFold)
    y_train_ml_np = y_train_ml.to_numpy()
    y_test_ml_np = y_test_ml.to_numpy()

    # --- E. Training & Evaluasi Model ---
    print("\n--- Memulai Pelatihan dan Evaluasi Model ---")

    # Daftar model
    models_to_evaluate_dl = {'CNN': create_cnn_model, 'LSTM': create_lstm_model, 'RNN': create_rnn_model}
    models_to_evaluate_ml = {
        'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
        'DecisionTree': DecisionTreeClassifier(random_state=42),
        'RandomForest': RandomForestClassifier(random_state=42),
        'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')
    }

    results = {}

    # --- Loop Pelatihan Model Deep Learning (Hanya Word2Vec) ---
    embedding_matrix = create_embedding_matrix_from_word2vec(w2v_model, word_index)

    for model_name, model_creator in models_to_evaluate_dl.items():
        print(f"\nMelatih model {model_name} dengan embedding Word2Vec...")
        kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        fold_accuracies = []

        for train, val in kfold.split(X_train_dl, np.argmax(y_train_dl, axis=1)):
            model = model_creator(embedding_matrix, max_len)
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
            model.fit(X_train_dl[train], y_train_dl[train],
                      epochs=20, batch_size=32,
                      validation_data=(X_train_dl[val], y_train_dl[val]),
                      callbacks=[early_stopping], verbose=0)
            _, acc = model.evaluate(X_train_dl[val], y_train_dl[val], verbose=0)
            fold_accuracies.append(acc)

        avg_cv_accuracy = np.mean(fold_accuracies)

        # Latih model final dan evaluasi di test set
        final_model = model_creator(embedding_matrix, max_len)
        final_model.fit(X_train_dl, y_train_dl, epochs=10, batch_size=32, verbose=0)
        y_pred_probs = final_model.predict(X_test_dl)
        y_pred = np.argmax(y_pred_probs, axis=1)
        y_true = np.argmax(y_test_dl, axis=1)

        accuracy = accuracy_score(y_true, y_pred)
        report = classification_report(y_true, y_pred, target_names=sentiment_mapping.keys(), zero_division=0)
        key = f"{model_name}_Word2Vec_DL" # Key unik untuk DL
        results[key] = {'model': final_model, 'accuracy': accuracy, 'report': report, 'avg_cv_accuracy': avg_cv_accuracy, 'type': 'dl'}

    # --- Loop Pelatihan Model Machine Learning (Word2Vec-Avg, BoW, TF-IDF) ---
    for feature_name, (X_train, X_test) in ml_feature_sets.items():
        for model_name, model_instance in models_to_evaluate_ml.items():
            print(f"\nMelatih model {model_name} dengan fitur {feature_name}...")
            kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
            fold_accuracies = []

            for train, val in kfold.split(X_train, y_train_ml_np):
                model = clone(model_instance)
                model.fit(X_train[train], y_train_ml_np[train])
                acc = model.score(X_train[val], y_train_ml_np[val])
                fold_accuracies.append(acc)

            avg_cv_accuracy = np.mean(fold_accuracies)

            # Latih model final dan evaluasi di test set
            final_model = clone(model_instance)
            final_model.fit(X_train, y_train_ml_np)
            y_pred = final_model.predict(X_test)

            accuracy = accuracy_score(y_test_ml_np, y_pred)
            report = classification_report(y_test_ml_np, y_pred, target_names=sentiment_mapping.keys(), zero_division=0)
            key = f"{model_name}_{feature_name}"

            # Tentukan tipe untuk implementasi nanti
            if feature_name == 'Word2Vec_Avg':
                model_type = 'ml_w2v'
            elif feature_name == 'BoW':
                model_type = 'ml_bow'
            else: # TFIDF
                model_type = 'ml_tfidf'

            results[key] = {'model': final_model, 'accuracy': accuracy, 'report': report, 'avg_cv_accuracy': avg_cv_accuracy, 'type': model_type}

    # --- F. Menampilkan Hasil dan Memilih Model Terbaik ---
    print("\n--- Hasil Evaluasi Akhir ---")
    best_model_key = None
    best_accuracy = 0
    for key, value in results.items():
        print(f"Kombinasi: {key} (Tipe: {value['type']})")
        print(f"  Akurasi pada Test Set: {value['accuracy']:.4f}")
        print(f"  Rata-rata Akurasi CV: {value['avg_cv_accuracy']:.4f}")
        print("  Laporan Klasifikasi:\n", value['report'])

        if value['accuracy'] > best_accuracy:
            best_accuracy = value['accuracy']
            best_model_key = key

    print(f"\nModel terbaik adalah: {best_model_key} dengan akurasi test set {best_accuracy:.4f}")

    # --- G. Implementasi Model Terbaik pada Data Sisa (70%) ---
    if best_model_key is None:
        print("\nTidak ada model yang berhasil dilatih. Implementasi dibatalkan.")
    else:
        best_model_data = results[best_model_key]
        best_model = best_model_data['model']
        best_model_type = best_model_data['type']

        print(f"\nMelakukan prediksi pada 70% data sisa menggunakan model: {best_model_key}")

        # Siapkan data 70% (sudah diproses di 'df_unannotated')

        if best_model_type == 'dl':
            # 1. Proses data 70% untuk DL
            unannotated_sequences = tokenizer.texts_to_sequences(df_unannotated['cleaned_text_nouns'])
            X_unannotated_feed = pad_sequences(unannotated_sequences, maxlen=max_len, padding='post', truncating='post')
            predictions_probs = best_model.predict(X_unannotated_feed)
            predictions = np.argmax(predictions_probs, axis=1)

        elif best_model_type == 'ml_w2v':
            # 2. Proses data 70% untuk ML Word2Vec Avg
            X_unannotated_feed = create_document_vectors(df_unannotated['cleaned_tokens_nouns'], w2v_model, w2v_model.vector_size)
            predictions = best_model.predict(X_unannotated_feed)

        elif best_model_type == 'ml_bow':
            # 3. Proses data 70% untuk ML BoW
            X_unannotated_feed = bow_vectorizer.transform(df_unannotated['cleaned_text_nouns'])
            predictions = best_model.predict(X_unannotated_feed)

        elif best_model_type == 'ml_tfidf':
            # 4. Proses data 70% untuk ML TF-IDF
            X_unannotated_feed = tfidf_vectorizer.transform(df_unannotated['cleaned_text_nouns'])
            predictions = best_model.predict(X_unannotated_feed)

        # Memetakan kembali hasil prediksi ke label sentimen
        inverse_sentiment_mapping = {v: k for k, v in sentiment_mapping.items()}
        df_unannotated[f'predicted_{target_aspect}'] = [inverse_sentiment_mapping.get(p, 'neutral') for p in predictions]

        # Menyimpan hasil prediksi
        output_filename = f'hasil_prediksi_{target_aspect}.csv'
        df_unannotated.to_csv(output_filename, index=False)

        print(f"Prediksi sentimen telah selesai.")
        print(f"Hasil disimpan di file: {output_filename}")

    print("\n--- Analisis Selesai ---")

NLTK stopwords sudah ada.
NLTK punkt_tab sudah ada.
NLTK POS Tagger (averaged_perceptron_tagger) sudah ada.
NLTK POS Tagger (averaged_perceptron_tagger_eng) sudah ada.
Persiapan awal selesai.
Jumlah data beranotasi: 3000
Jumlah data belum beranotasi: 7747
Menerapkan prapemrosesan (Tokenize, POS Tag, Filter Nouns, Stopwords)...

--- Menyiapkan Model Embedding (Word2Vec) ---
Model Word2Vec selesai dilatih.

--- Mempersiapkan Fitur Set Machine Learning ---
Fitur Word2Vec (Avg) siap.
Fitur Bag of Words (BoW) siap.
Fitur TF-IDF siap.

--- Memulai Pelatihan dan Evaluasi Model ---

Melatih model CNN dengan embedding Word2Vec...
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step

Melatih model LSTM dengan embedding Word2Vec...
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step

Melatih model RNN dengan embedding Word2Vec...
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step

Melatih model LogisticRegression dengan fitur Word2Vec_Avg...

Melatih model DecisionTree dengan fitur Word2Vec_Avg...

Melatih model RandomForest dengan

In [46]:
pd.set_option('display.max_colwidth', None)
df_annotated[df_annotated['event_sentiment'] == 0].head(3)

,text,created_at,event_sentiment,circuit_sentiment,cleaned_tokens_nouns,cleaned_text_nouns
1,Legenda balap motor Valentino Rossi tiba di Jakarta jelang penyelenggaraan MotoGP Mandalika 2025 pada akhir pekan mendatang.\n\nRossi telah mendarat di Bandar Udara Soekarno-Hatta pada Senin (29/9) pukul 22.30 WIB. Sosok yang punya julukan The Doctor itu mendapat sambutan hangat di bandara.\n\nKedatangan Rossi sudah ditunggu-tunggu oleh fans. Bahkan ada yang membawa helm untuk ditandatangani pembalap yang punya sembilan gelar juara dunia itu.,2025-09-30 20.54.12,0,1,"[legenda, balap, motor, valentino, rossi, tiba, jakarta, jelang, selenggara, motogp, akhir, pekan, datang, rossi, darat, bandar, soekarnohatta, senin, pukul, wib, sosok, punya, doctor, sambut, hangat, bandara, datang, rossi, ditunggutunggu, fans, bawa, helm, ditandatangani, balap, punya, sembilan, juara, dunia]",legenda balap motor valentino rossi tiba jakarta jelang selenggara motogp akhir pekan datang rossi darat bandar soekarnohatta senin pukul wib sosok punya doctor sambut hangat bandara datang rossi ditunggutunggu fans bawa helm ditandatangani balap punya sembilan juara dunia
3,Menteri Pemuda dan Olahraga Erick Thohir menyebutkan jumlah penonton MotoGP Indonesia di Sirkuit Mandalika Kabupaten Lombok Tengah Nusa Tenggara Barat tembus sampai 142 ribu orang.https://t.co/veteTzY0ts,2025-10-05 12.39.53,0,0,"[menteri, pemuda, thohir, sebut, motogp, indonesia, sirkuit, tengah, nusa, tenggara, barat, tembus, ribu, orang]",menteri pemuda thohir sebut motogp indonesia sirkuit tengah nusa tenggara barat tembus ribu orang
4,Momen Pacu Jalur Rayyan Arkan Dikha Guncang Mandalika MotoGP Indonesia\n\n#TVRI #TVRINasional #MediaPemersatuBangsa,2025-10-04 20.21.14,0,1,"[momen, jalur, rayyan, kha, guncang, motogp, indonesia]",momen jalur rayyan kha guncang motogp indonesia


In [47]:
df_annotated[df_annotated['event_sentiment'] == 1].head(3)

,text,created_at,event_sentiment,circuit_sentiment,cleaned_tokens_nouns,cleaned_text_nouns
0,"""Five years ago, I had my most serious accident, not in absolute terms, but because of the wrong choices I made. I came back too early, and then everything got worse.\n\nIn those three or four years when I missed the chance to fight for the world championship, I learned a lot about my life. And so it changed my character a bit. I'm proud to continue growing in life."" : Marc Marquez \n\nSource: TNT Sports",2025-07-20 14.40.42,1,1,"[years, accident, terms, choices, everything, years, chance, world, championship, lot, life, character, bit, proud, life, marc, marquez, source, tnt, sports]",years accident terms choices everything years chance world championship lot life character bit proud life marc marquez source tnt sports
2,Menyusuri Sejarah Motorsport Indonesia di Museum MotoGP Mandalika https://www.idntimes.com/sport/arena/menyusuri-sejarah-motorsport-indonesia-di-museum-motogp-mandalika-00-dtjm6-z3zvvk?utm_source=threads&utm_medium=threads&utm_campaign=dlvr.it,2025-10-06 19.08.19,1,1,"[susur, sejarah, motorsport, indonesia, museum, motogp]",susur sejarah motorsport indonesia museum motogp
5,"Ini sih bukan Circuit terbaik di lokal area tapi udah men-Dunia, asli gak ada obat gila²..",2022-11-09 08.51.31,1,0,"[sih, bukan, circuit, baik, area, dunia, asli, gak, obat, gila]",sih bukan circuit baik area dunia asli gak obat gila


In [48]:
df_annotated[df_annotated['event_sentiment'] == 2].head(3)

,text,created_at,event_sentiment,circuit_sentiment,cleaned_tokens_nouns,cleaned_text_nouns
39,Marc Marquez terjatuh di balapatan utama MotoGP Mandalika 2025. Dokter menyatakan Marquez kemungkinan mengalami fraktur tulang selangka. Baca di sini: https://t.co/dUjrk4uz7X / #MotoGP #MarcMarquez https://t.co/ao3DKbR6Iv,2025-10-05 10.13.45,2,1,"[marc, marquez, jatuh, balapatan, motogp, dokter, nyata, alami, fraktur, tulang, selangka, baca, sini]",marc marquez jatuh balapatan motogp dokter nyata alami fraktur tulang selangka baca sini
46,Marc Marquez selalu jatuh di sirkuit mandalika di 3 kali epidose motogp indonesia https://t.co/ZBwCgfoJQf,2025-10-03 13.27.43,2,2,"[marc, marquez, selalu, jatuh, sirkuit, indonesia]",marc marquez selalu jatuh sirkuit indonesia
62,"🏍️ Marc Marquez Cedera di Sirkuit Mandalika!\nKecelakaan di lap pertama MotoGP Indonesia 2025 membuat bintang Ducati itu alami patah tulang selangka kanan. Ia kini diterbangkan ke Madrid untuk perawatan lanjutan — musimnya pun terancam berakhir lebih cepat! 😢\n\n💬 Marquez: ""Saya sedih, tapi ini bagian dari risiko seorang pembalap.""\n\nBaca kronologi lengkap dan update kondisi terbaru Marquez hanya di 👉\n🌐 updatenusantara.com \n\n#MotoGPMandalika #MarcMarquez #Ducati #MotoGP2025 #UpdateNusantara",2025-10-05 19.25.51,2,2,"[marc, marquez, cedera, sirkuit, celaka, pertama, motogp, indonesia, buat, bintang, ducati, alami, patah, tulang, selangka, kanan, kini, terbang, awat, lanjut, musim, ancam, akhir, lebih, cepat, marquez, sedih, risiko, orang, balap, baca, kronologi, kondisi, baru, marquez, updatenusantaracom]",marc marquez cedera sirkuit celaka pertama motogp indonesia buat bintang ducati alami patah tulang selangka kanan kini terbang awat lanjut musim ancam akhir lebih cepat marquez sedih risiko orang balap baca kronologi kondisi baru marquez updatenusantaracom


In [49]:
df_annotated[df_annotated['circuit_sentiment'] == 0].head(3)

,text,created_at,event_sentiment,circuit_sentiment,cleaned_tokens_nouns,cleaned_text_nouns
3,Menteri Pemuda dan Olahraga Erick Thohir menyebutkan jumlah penonton MotoGP Indonesia di Sirkuit Mandalika Kabupaten Lombok Tengah Nusa Tenggara Barat tembus sampai 142 ribu orang.https://t.co/veteTzY0ts,2025-10-05 12.39.53,0,0,"[menteri, pemuda, thohir, sebut, motogp, indonesia, sirkuit, tengah, nusa, tenggara, barat, tembus, ribu, orang]",menteri pemuda thohir sebut motogp indonesia sirkuit tengah nusa tenggara barat tembus ribu orang
5,"Ini sih bukan Circuit terbaik di lokal area tapi udah men-Dunia, asli gak ada obat gila²..",2022-11-09 08.51.31,1,0,"[sih, bukan, circuit, baik, area, dunia, asli, gak, obat, gila]",sih bukan circuit baik area dunia asli gak obat gila
6,Nonton MotoGP tiket Grandstand A. Pengalaman yang sangat memuaskan dan seru. Balapannya banyak yang crash tapi aman karena tidak ada yang fatal. Panasnya kuta mandalika bikin meleleh untung saya beli tiket di stand yang ada rooftopnya. Tapi harganya lumayan mahal dan gak sempat ambil bonus hospitalitynya dan nonton festival band terkenal karena harus ngejar kapal ke Bali.,2024-11-08 08.51.31,0,0,"[nonton, motogp, tiket, alam, sangat, muas, seru, balap, banyak, crash, panas, kuta, leleh, beli, tiket, rooftopnya, harga, gak, sempat, bonus, hospitalitynya, festival, ngejar, bal]",nonton motogp tiket alam sangat muas seru balap banyak crash panas kuta leleh beli tiket rooftopnya harga gak sempat bonus hospitalitynya festival ngejar bal


In [50]:
df_annotated[df_annotated['circuit_sentiment'] == 1].head(3)

,text,created_at,event_sentiment,circuit_sentiment,cleaned_tokens_nouns,cleaned_text_nouns
0,"""Five years ago, I had my most serious accident, not in absolute terms, but because of the wrong choices I made. I came back too early, and then everything got worse.\n\nIn those three or four years when I missed the chance to fight for the world championship, I learned a lot about my life. And so it changed my character a bit. I'm proud to continue growing in life."" : Marc Marquez \n\nSource: TNT Sports",2025-07-20 14.40.42,1,1,"[years, accident, terms, choices, everything, years, chance, world, championship, lot, life, character, bit, proud, life, marc, marquez, source, tnt, sports]",years accident terms choices everything years chance world championship lot life character bit proud life marc marquez source tnt sports
1,Legenda balap motor Valentino Rossi tiba di Jakarta jelang penyelenggaraan MotoGP Mandalika 2025 pada akhir pekan mendatang.\n\nRossi telah mendarat di Bandar Udara Soekarno-Hatta pada Senin (29/9) pukul 22.30 WIB. Sosok yang punya julukan The Doctor itu mendapat sambutan hangat di bandara.\n\nKedatangan Rossi sudah ditunggu-tunggu oleh fans. Bahkan ada yang membawa helm untuk ditandatangani pembalap yang punya sembilan gelar juara dunia itu.,2025-09-30 20.54.12,0,1,"[legenda, balap, motor, valentino, rossi, tiba, jakarta, jelang, selenggara, motogp, akhir, pekan, datang, rossi, darat, bandar, soekarnohatta, senin, pukul, wib, sosok, punya, doctor, sambut, hangat, bandara, datang, rossi, ditunggutunggu, fans, bawa, helm, ditandatangani, balap, punya, sembilan, juara, dunia]",legenda balap motor valentino rossi tiba jakarta jelang selenggara motogp akhir pekan datang rossi darat bandar soekarnohatta senin pukul wib sosok punya doctor sambut hangat bandara datang rossi ditunggutunggu fans bawa helm ditandatangani balap punya sembilan juara dunia
2,Menyusuri Sejarah Motorsport Indonesia di Museum MotoGP Mandalika https://www.idntimes.com/sport/arena/menyusuri-sejarah-motorsport-indonesia-di-museum-motogp-mandalika-00-dtjm6-z3zvvk?utm_source=threads&utm_medium=threads&utm_campaign=dlvr.it,2025-10-06 19.08.19,1,1,"[susur, sejarah, motorsport, indonesia, museum, motogp]",susur sejarah motorsport indonesia museum motogp


In [51]:
df_annotated[df_annotated['circuit_sentiment'] == 2].head(3)

,text,created_at,event_sentiment,circuit_sentiment,cleaned_tokens_nouns,cleaned_text_nouns
19,"MotoGP Mandalika 2023. Sudah banyak perubahan kemajuan dari tahun lalu. Tahun lalu nnton di zone B, skrg di zone J di tikungan maut 😃",2023-10-15 19.55.51,0,2,"[motogp, banyak, maju, tahun, lalu, tahun, nnton, zone, skrg, tikung, maut]",motogp banyak maju tahun lalu tahun nnton zone skrg tikung maut
46,Marc Marquez selalu jatuh di sirkuit mandalika di 3 kali epidose motogp indonesia https://t.co/ZBwCgfoJQf,2025-10-03 13.27.43,2,2,"[marc, marquez, selalu, jatuh, sirkuit, indonesia]",marc marquez selalu jatuh sirkuit indonesia
48,di mandalika itu yang jahat anginnya yang udah pernah ke sana dan nyoba treknya pasti tau deh hahahaha,2025-10-03 12.06.46,1,2,"[pernah, sana, trek, tau, deh, hahahaha]",pernah sana trek tau deh hahahaha


In [55]:
pd.set_option('display.max_colwidth', None)
df_unannotated[df_unannotated['predicted_event_sentiment'] == "positive"].head(3)

,text,created_at,cleaned_tokens_nouns,cleaned_text_nouns,predicted_event_sentiment,predicted_circuit_sentiment
1,"Kemarin liat berita Mandalika GP, tiketnya gak habis terjual, yg laku cuma sekitar 30%. Sementara sore ini di tv, denger penyiar motoGP yg dikit2 puji alam sekitar sirkuit yg Indah, tanding di sini selalu menyenangkan, jadi kepikiran… Orang2 kita kenapa gak terlalu minat nonton GP di sini ? Selain karena tiket pesawat domestik yg mahal ?",2025-10-03 15.36.02,"[kemarin, liat, berita, tiket, gak, habis, laku, cuma, sore, denger, dikit, puji, alam, sirkuit, indah, sini, selalu, senang, jadi, pikir, orang, gak, terlalu, sini, tiket, pesawat, domestik, mahal]",kemarin liat berita tiket gak habis laku cuma sore denger dikit puji alam sirkuit indah sini selalu senang jadi pikir orang gak terlalu sini tiket pesawat domestik mahal,positive,positive
3,"Dua tahun berturut-turut nonton MotoGP Mandalika, rasanya nggak pernah bosan. Euforianya selalu bikin merinding, experience-nya selalu terkenang. Mandalika makin ke sini makin berasa upgrade vibes-nya 🔥 Gila sih, meskipun rintangannya harus menerjang panasnya Mandalika yg bikin keleyengan.. tapi experience nya gabisa dilupain!!!",2025-10-07 18.15.24,"[tahun, motogp, rasa, pernah, bosan, euforia, selalu, selalu, kenang, sini, asa, vibesnya, gila, sih, meski, rintang, terjang, panas, experience, nya, gabisa, dilupain]",tahun motogp rasa pernah bosan euforia selalu selalu kenang sini asa vibesnya gila sih meski rintang terjang panas experience nya gabisa dilupain,positive,positive
5,1 days to go! ⏰ 🏁\n\nReady to witness history at MotoGP Mandalika 2025? Take a pit stop at @themandalikagp to find out more🔥\n\n#WonderfulIndonesia #IndonesianGP #MotoGP2025 #PertaminaGrandPrixofIndonesia #MandalikaCircuit #SirkuitMandalika #Mandalika,2025-10-02 12.33.10,"[days, history, pit, stop]",days history pit stop,positive,neutral


In [56]:
pd.set_option('display.max_colwidth', None)
df_unannotated[df_unannotated['predicted_event_sentiment'] == "neutral"].head(3)

,text,created_at,cleaned_tokens_nouns,cleaned_text_nouns,predicted_event_sentiment,predicted_circuit_sentiment
0,"Menggenang marco simoncelli:\nPrompt :👇☠️\nBuatlah foto ini dia sedang duduk diatas motor balap ducati berwarna putih nomor 58 berwarna merah motogp. lengkap dengan seragam balap. dengan wajah santai menatap kedepan dia memakai kaca mata ribeng hitam,\ndan di sampingnya ada Valentino rossi dengan motor balap ducati 2025nya lengkap dengan baju balap,\nbuat juga masing masing ada umbrella girl yg berdiri di samping pembalap, suasananya berada di tengah arena balap sikuit mandalika 2025 dangan latar",2025-10-03 22.34.55,"[genang, marco, simoncelli, prompt, buat, foto, sedang, duduk, atas, motor, ducati, warna, putih, nomor, motogp, lengkap, balap, santai, tatap, depan, pakai, kaca, mata, ribeng, hitam, valentino, rossi, motor, balap, ducati, lengkap, baju, balap, buat, girl, diri, suasana, tengah, arena, balap, sikuit, dang, latar]",genang marco simoncelli prompt buat foto sedang duduk atas motor ducati warna putih nomor motogp lengkap balap santai tatap depan pakai kaca mata ribeng hitam valentino rossi motor balap ducati lengkap baju balap buat girl diri suasana tengah arena balap sikuit dang latar,neutral,neutral
2,Ternyata MotoGP Mandalika barengan sama F1 Singapura. Kalau ada bujetnya mending ntn lgsg yang mana dan kenapa?,2025-10-04 17.11.57,"[nyata, motogp, bareng, sama, singapura, bujet, lgsg, mana]",nyata motogp bareng sama singapura bujet lgsg mana,neutral,neutral
4,"Welcome back to Paradise!\nMotoGP Mandalika, proud of you Lombok…\n#lombok #mandalikacircuit",2024-09-28 07.02.23,"[motogp, proud]",motogp proud,neutral,neutral


In [57]:
pd.set_option('display.max_colwidth', None)
df_unannotated[df_unannotated['predicted_event_sentiment'] == "negative"].head(3)

,text,created_at,cleaned_tokens_nouns,cleaned_text_nouns,predicted_event_sentiment,predicted_circuit_sentiment
33,"Realita absurd\n🇮🇩 ke Mandalika 5jt\n🇲🇾 ke sepang 700rb\nSama-sama tempat nonton balapan, tapi kok rasanya dompet di Mandalika lebih berat dari pada nonton di Sepang 😭",2025-09-23 11.56.40,"[realita, absurd, sepang, samasama, balap, kok, rasa, dompet, berat, sepang]",realita absurd sepang samasama balap kok rasa dompet berat sepang,negative,neutral
36,Marco Bezzecchi sudah kembali ke garasi pasca insiden crash dengan Marc Marquez.\n\n📸 GPRI\n\n#GPRacingIndonesia #GPRI #RespectAllRiders #MotoGP #IndonesianGP #mandalika,2025-10-05 14.14.49,"[marco, bezzecchi, garasi, pasca, crash, marquez, gpri]",marco bezzecchi garasi pasca crash marquez gpri,negative,negative
51,Seputaran MotoGP Mandalika 2025 pren to pren.. kadang ga penting event nya… lebih penting seseruannya..,2025-10-28 20.49.19,"[putar, motogp, pren, event, nya, lebih, seru]",putar motogp pren event nya lebih seru,negative,neutral


In [14]:
pd.set_option('display.max_colwidth', None)
df_unannotated[df_unannotated['predicted_circuit_sentiment'] == "positive"].head(3)

,text,created_at,cleaned_tokens_nouns,cleaned_text_nouns,predicted_event_sentiment,predicted_circuit_sentiment
1,"Kemarin liat berita Mandalika GP, tiketnya gak habis terjual, yg laku cuma sekitar 30%. Sementara sore ini di tv, denger penyiar motoGP yg dikit2 puji alam sekitar sirkuit yg Indah, tanding di sini selalu menyenangkan, jadi kepikiran… Orang2 kita kenapa gak terlalu minat nonton GP di sini ? Selain karena tiket pesawat domestik yg mahal ?",2025-10-03 15.36.02,"['kemarin', 'liat', 'berita', 'tiket', 'gak', 'habis', 'laku', 'cuma', 'sore', 'denger', 'dikit', 'puji', 'alam', 'sirkuit', 'indah', 'sini', 'selalu', 'senang', 'jadi', 'pikir', 'orang', 'gak', 'terlalu', 'sini', 'tiket', 'pesawat', 'domestik', 'mahal']",kemarin liat berita tiket gak habis laku cuma sore denger dikit puji alam sirkuit indah sini selalu senang jadi pikir orang gak terlalu sini tiket pesawat domestik mahal,positive,positive
3,"Dua tahun berturut-turut nonton MotoGP Mandalika, rasanya nggak pernah bosan. Euforianya selalu bikin merinding, experience-nya selalu terkenang. Mandalika makin ke sini makin berasa upgrade vibes-nya 🔥 Gila sih, meskipun rintangannya harus menerjang panasnya Mandalika yg bikin keleyengan.. tapi experience nya gabisa dilupain!!!",2025-10-07 18.15.24,"['tahun', 'motogp', 'rasa', 'pernah', 'bosan', 'euforia', 'selalu', 'selalu', 'kenang', 'sini', 'asa', 'vibesnya', 'gila', 'sih', 'meski', 'rintang', 'terjang', 'panas', 'experience', 'nya', 'gabisa', 'dilupain']",tahun motogp rasa pernah bosan euforia selalu selalu kenang sini asa vibesnya gila sih meski rintang terjang panas experience nya gabisa dilupain,positive,positive
29,"DAY 2 MOTOGP 2025 🤩🏁✨\n— rame banget, kerennn🔥\n\nHair ini yang kita tunggu tunggu guys. Ayo ayo berangkat ke Mandalika sekarang! See you at Pertamina Mandalika International Circuit ❤️\n\n#indonesiangp #burnthelimits #indonesiamendunia #motogp #mandalika #lombok",2025-10-05 06.55.41,"['day', 'rame', 'banget', 'kerennn', 'hair', 'tunggu', 'guys', 'berangkat', 'sekarang', 'pertamina', 'circuit']",day rame banget kerennn hair tunggu guys berangkat sekarang pertamina circuit,neutral,positive


In [15]:
pd.set_option('display.max_colwidth', None)
df_unannotated[df_unannotated['predicted_circuit_sentiment'] == "neutral"].head(3)

,text,created_at,cleaned_tokens_nouns,cleaned_text_nouns,predicted_event_sentiment,predicted_circuit_sentiment
0,"Menggenang marco simoncelli:\nPrompt :👇☠️\nBuatlah foto ini dia sedang duduk diatas motor balap ducati berwarna putih nomor 58 berwarna merah motogp. lengkap dengan seragam balap. dengan wajah santai menatap kedepan dia memakai kaca mata ribeng hitam,\ndan di sampingnya ada Valentino rossi dengan motor balap ducati 2025nya lengkap dengan baju balap,\nbuat juga masing masing ada umbrella girl yg berdiri di samping pembalap, suasananya berada di tengah arena balap sikuit mandalika 2025 dangan latar",2025-10-03 22.34.55,"['genang', 'marco', 'simoncelli', 'prompt', 'buat', 'foto', 'sedang', 'duduk', 'atas', 'motor', 'ducati', 'warna', 'putih', 'nomor', 'motogp', 'lengkap', 'balap', 'santai', 'tatap', 'depan', 'pakai', 'kaca', 'mata', 'ribeng', 'hitam', 'valentino', 'rossi', 'motor', 'balap', 'ducati', 'lengkap', 'baju', 'balap', 'buat', 'girl', 'diri', 'suasana', 'tengah', 'arena', 'balap', 'sikuit', 'dang', 'latar']",genang marco simoncelli prompt buat foto sedang duduk atas motor ducati warna putih nomor motogp lengkap balap santai tatap depan pakai kaca mata ribeng hitam valentino rossi motor balap ducati lengkap baju balap buat girl diri suasana tengah arena balap sikuit dang latar,neutral,neutral
2,Ternyata MotoGP Mandalika barengan sama F1 Singapura. Kalau ada bujetnya mending ntn lgsg yang mana dan kenapa?,2025-10-04 17.11.57,"['nyata', 'motogp', 'bareng', 'sama', 'singapura', 'bujet', 'lgsg', 'mana']",nyata motogp bareng sama singapura bujet lgsg mana,neutral,neutral
4,"Welcome back to Paradise!\nMotoGP Mandalika, proud of you Lombok…\n#lombok #mandalikacircuit",2024-09-28 07.02.23,"['motogp', 'proud']",motogp proud,neutral,neutral


In [16]:
pd.set_option('display.max_colwidth', None)
df_unannotated[df_unannotated['predicted_circuit_sentiment'] == "negative"].head(3)

,text,created_at,cleaned_tokens_nouns,cleaned_text_nouns,predicted_event_sentiment,predicted_circuit_sentiment
17,"MASYARAKAT Australia minta MotoGP Mandalika dihapus dari kalender MotoGP efek cedera Marc Marquez. Sejatinya, bukan faktor Marquez yang jadi permasalahan, namun kondisi gravel di Sirkuit Mandalika yang dinilai berbahaya sehingga membuat sang juara dunia MotoGP 2025 itu mengalami cedera parah.",2025-10-10 15.24.23,"['masyarakat', 'motogp', 'hapus', 'kalender', 'motogp', 'efek', 'cedera', 'marc', 'marquez', 'sejati', 'bukan', 'faktor', 'marquez', 'jadi', 'masalah', 'kondisi', 'gravel', 'sirkuit', 'bahaya', 'buat', 'juara', 'dunia', 'motogp', 'alami', 'cedera', 'parah']",masyarakat motogp hapus kalender motogp efek cedera marc marquez sejati bukan faktor marquez jadi masalah kondisi gravel sirkuit bahaya buat juara dunia motogp alami cedera parah,neutral,negative
36,Marco Bezzecchi sudah kembali ke garasi pasca insiden crash dengan Marc Marquez.\n\n📸 GPRI\n\n#GPRacingIndonesia #GPRI #RespectAllRiders #MotoGP #IndonesianGP #mandalika,2025-10-05 14.14.49,"['marco', 'bezzecchi', 'garasi', 'pasca', 'crash', 'marquez', 'gpri']",marco bezzecchi garasi pasca crash marquez gpri,negative,negative
75,"masalahnya juga mayoritas masyarakat nya begitu, contoh lain nya ya diluar topi, kemarin di balapan motogp mandalika banyak rider yg ngeluh soal keamanan gravel sirkuit yg dinilai berbahaya, dan isi komennya masyarakat nya malah hujat para rider dan terlalu buta akan kritikan, mayoritas pejabat di indo cerminan mayoritas masyarakatnya",2025-10-27 20.20.37,"['masalah', 'mayoritas', 'contoh', 'luar', 'topi', 'kemarin', 'balap', 'motogp', 'banyak', 'rider', 'soal', 'aman', 'gravel', 'sirkuit', 'bahaya', 'isi', 'komennya', 'masyarakat', 'malah', 'hujat', 'rider', 'terlalu', 'buta', 'kritik', 'mayoritas', 'jabat', 'cermin', 'mayoritas']",masalah mayoritas contoh luar topi kemarin balap motogp banyak rider soal aman gravel sirkuit bahaya isi komennya masyarakat malah hujat rider terlalu buta kritik mayoritas jabat cermin mayoritas,negative,negative


In [30]:
df_annotated['event_sentiment'].value_counts()

,count
event_sentiment,
1,1451
0,1178
2,371


In [31]:
df_annotated['circuit_sentiment'].value_counts()

,count
circuit_sentiment,
1,2257
0,438
2,305


In [32]:
df_unannotated['predicted_event_sentiment'].value_counts()

,count
predicted_event_sentiment,
neutral,4410
positive,2785
negative,552


In [33]:
df_unannotated['predicted_circuit_sentiment'].value_counts()

,count
predicted_circuit_sentiment,
neutral,6624
positive,714
negative,409


In [35]:
df_annotated.to_csv('/content/drive/My Drive/Kuliah MTI/Anmedsos/df_annotated_clean_full.csv', sep=';', index=False)

In [3]:
df_unannotated.to_csv('/content/drive/My Drive/Kuliah MTI/Anmedsos/df_unannotated_prediction_full.csv', sep=';', index=False)

NameError: name 'df_unannotated' is not defined

In [12]:
import pandas as pd
df_annotated = pd.read_csv('/content/drive/My Drive/Kuliah MTI/Anmedsos/df_annotated_clean_full.csv', sep=';')

In [13]:
import pandas as pd
df_unannotated = pd.read_csv('/content/drive/My Drive/Kuliah MTI/Anmedsos/df_unannotated_prediction_full.csv', sep=';')